<a href="https://colab.research.google.com/github/sipocz/DataMiner/blob/main/DataMiner_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import os
from bs4 import  BeautifulSoup

In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
path="/content/drive/MyDrive/001_METU/2. félév/dolgozat/adatok/"

In [6]:
fname="Eseményfeladatok _ RG - IT Service Management.html"

In [10]:
htmltext=open(path+fname)

In [12]:
ht=htmltext.read()

In [83]:
print("------------------------------------------------")

soup=BeautifulSoup(ht,"html.parser")
table_rows=soup.find_all("tr", class_="list_row list_odd is-selected")


------------------------------------------------


In [84]:
table_rows[1]

<tr class="list_row list_odd is-selected" collapsed="true" data-list_id="incident_task" data-type="list2_row" data-updated-on="2021-10-19 13:07:54" id="row_incident_task_f6954458db5b74549fdbbb13f3961903" record_class="incident_task" style="" sys_id="f6954458db5b74549fdbbb13f3961903"><td class="list_decoration_cell col-control col-small col-center " rowspan="1" style="white-space:nowrap;"><span class="input-group-checkbox"><input class="checkbox " data-list_id="incident_task" data-type="list2_checkbox" id="check_incident_task_f6954458db5b74549fdbbb13f3961903" name="check_incident_task" onblur="window.NOW.accessibility.onBlurRemoveMSHCOutline('check_container_incident_task_f6954458db5b74549fdbbb13f3961903')" onfocus="window.NOW.accessibility.onFocusAddMSHCOutline('check_container_incident_task_f6954458db5b74549fdbbb13f3961903')" title="Rekord megjelölése listaművelethez" type="checkbox"/><label class="checkbox-label" for="check_incident_task_f6954458db5b74549fdbbb13f3961903" id="check_co

In [85]:
a_row=table_rows[2].find_all("td",class_="vt")

In [86]:
a_row

[<td class="vt" ng-non-bindable=""><a aria-label="Nyitott rekord: TASK0230843" class="linked formlink" href="https://richterprod.service-now.com/incident_task.do?sys_id=179f13eadb63bc909fdbbb13f3961908&amp;sysparm_record_target=incident_task&amp;sysparm_record_row=5&amp;sysparm_record_rows=224&amp;sysparm_record_list=assignment_group.name%3DMAS_DOR%5EORDERBYsys_created_on">TASK0230843</a></td>,
 <td class="vt" ng-non-bindable="">AZ RG-LAB rendszerből nem tudunk feladatot nyomtatni, hibaüzenet jelenik meg.</td>,
 <td class="vt" ng-non-bindable="">3 - Közepes</td>,
 <td class="vt" ng-non-bindable="">Lezárt (befejezett)</td>,
 <td class="vt" ng-non-bindable=""><a aria-label="Nyitott rekord: MAS_DOR" class="linked" href="https://richterprod.service-now.com/sys_user_group.do?sys_id=1530f153db837050f8f6345239961925" sys_id="1530f153db837050f8f6345239961925">MAS_DOR</a></td>,
 <td class="vt" ng-non-bindable=""><a aria-label="Nyitott rekord: Károly Zoltán Magyar" class="linked" href="https://r

In [108]:
a_row[10]["title"]

'A következő beírás került a műszaknaplóba.-> Szteroid II: Kromatográfiás L5 légkez. split range szabályzás rendellenesen működik (13 °C-os befújt levegő hőfok és 22 °C-os alapjel mellett a hűtési szelep volt még nyitva, a rendszer nagyon lassan szabályzott). Hűtés letiltva. Amint a szabályozás átváltott fűtésbe, normális sebességgel kezdett működni. Kérjük a jelenség kivizsgálását, javítását.'

In [130]:
def print_a_row(a_row,to_print=False):
    task_id=a_row[0].a["aria-label"].split(": ")[1]
    task_link=a_row[0].a["href"]
    task_description=a_row[1].text
    inc_name=a_row[6].a["aria-label"].split(": ")[1]
    inc_link=a_row[6].a["href"]
    t_creation=a_row[8].div["timeago"]
    t_start=a_row[7].div["timeago"]
    t_end=a_row[9].div["timeago"]
    try:
        full_text=a_row[10]["title"]
    except:
        full_text="None"

    if to_print:
        print("taskid:",task_id,"\ntask_link: ",task_link)
        print(a_row[1].text)

        print("incident name:",inc_name)
        print("incident link:",inc_link )
    
        print("Creation time:",t_creation)
        print("Start time:",t_start)
        print("End time",t_end)
    


         
        print("Text descriptiopn:",full_text)
        
        print("__________________________")

In [126]:
table_rows[0]

<tr class="list_row list_odd is-selected" collapsed="true" data-list_id="incident_task" data-type="list2_row" data-updated-on="2021-11-05 06:09:23" id="row_incident_task_fa348d90dbaa7090b4c40573f39619c5" record_class="incident_task" style="" sys_id="fa348d90dbaa7090b4c40573f39619c5"><td class="list_decoration_cell col-control col-small col-center " rowspan="1" style="white-space:nowrap;"><span class="input-group-checkbox"><input class="checkbox " data-list_id="incident_task" data-type="list2_checkbox" id="check_incident_task_fa348d90dbaa7090b4c40573f39619c5" name="check_incident_task" onblur="window.NOW.accessibility.onBlurRemoveMSHCOutline('check_container_incident_task_fa348d90dbaa7090b4c40573f39619c5')" onfocus="window.NOW.accessibility.onFocusAddMSHCOutline('check_container_incident_task_fa348d90dbaa7090b4c40573f39619c5')" title="Rekord megjelölése listaművelethez" type="checkbox"/><label class="checkbox-label" for="check_incident_task_fa348d90dbaa7090b4c40573f39619c5" id="check_co

In [133]:
for a_row_e in table_rows:
    a_row=a_row_e.find_all("td",class_="vt")
    print_a_row(a_row,to_print=True)


taskid: TASK0216008 
task_link:  https://richterprod.service-now.com/incident_task.do?sys_id=fa348d90dbaa7090b4c40573f39619c5&sysparm_record_target=incident_task&sysparm_record_row=1&sysparm_record_rows=224&sysparm_record_list=assignment_group.name%3DMAS_DOR%5EORDERBYsys_created_on
Az OV-01 TFL operátori állomás meghibásodott, nem indítható.
incident name: INC0118840
incident link: https://richterprod.service-now.com/incident.do?sys_id=ece4a22bdb5ab850b4c40573f3961976
Creation time: 2021-09-08 05:45:29
Start time: 2021-09-08 05:48:13
End time 2021-11-05 06:09:23
Text descriptiopn: Az operátori állomás meghibásodott, nem indítható, a rendszerhez tartozó berendezések nem használható, sürgős javítást, segítséget kérünk, mert a program szerint du. tervezett indulás van.

" Imre Váradi
Work notes•2021-09-06 12:30:06
A OW01-TFL állomás leszállításra került a MASDOR demo helyiségbe, tesztelés, esemény elemzés céljából. A szerver sikeresen elindult. Az eseménynapló alapján egy kritikus rendsze